In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import util

pd.set_option('display.float_format', '{:,.1%}'.format)

In [2]:
# add trip type to trip tables
def prep_trip_df(trip):
    """Mode shares will represent shares across Drive Alone, Shared Ride, 
       public transit, and walk and bike trips only. 
    """
    df_trip = trip.copy()
    df_trip.rename(columns={'mode':'Mode'}, inplace=True)
    df_trip['Trip Type'] = 'Non-Work'
    df_trip.loc[df_trip['dpurp'] == 'Work', 'Trip Type'] = 'Work'

    # add all tripa
    df = df_trip.copy()
    df['Trip Type'] = 'All Trips'
    df_trip = pd.concat([df_trip.sort_values('Trip Type'), df], ignore_index=True)

    # Exclude TNC, School Bus, and Other modes
    df_trip = df_trip[~df_trip['Mode'].isin(['TNC', 'School Bus', 'Other'])]

    # Aggregate modes
    df_trip['Mode'] = df_trip['Mode'].replace({
        'SOV': 'Drove Alone',
        'HOV2': 'Shared Ride',
        'HOV3+': 'Shared Ride',
        'Transit': 'Transit',
        'Walk': 'Walk',
        'Bike': 'Bike',
    })

    return df_trip

In [3]:
person = pd.read_csv(util.output_path / 'agg/dash/person_geog.csv')

# counties
trip_county = prep_trip_df(pd.read_csv(util.output_path / 'agg/dash/mode_share_county.csv'))
# remove "Outside Region"
trip_county = trip_county[trip_county['hh_county'] != 'Outside Region']
# regional geographies
trip_rg = prep_trip_df(pd.read_csv(util.output_path / 'agg/dash/mode_share_rg.csv'))
# centers
trip_rgc = pd.read_csv(util.output_path / 'agg/dash/mode_share_rgc.csv')
# add all RGCs
df = trip_rgc[trip_rgc['hh_rgc'] != 'Not in RGC'].groupby(['dpurp','mode']).sum()[['trexpfac']].reset_index()
df['hh_rgc'] = 'All RGCs'
trip_rgc = pd.concat([df, trip_rgc])
trip_rgc = prep_trip_df(trip_rgc)

In [4]:
# equity geographies
equity_geogs = util.summary_config['hh_equity_geogs']
trip_equity_geog = pd.DataFrame()
for geog in equity_geogs:
    df = prep_trip_df(pd.read_csv(util.output_path/ ('agg/dash/mode_share_'+geog+'.csv')))
    df['equity_geog'] = df[geog].map({
                                0: 'Below Regional Average', 
                                1: 'Above Regional Average', 
                                2: 'Higher Share of Equity Population'})
    
    trip_equity_geog = pd.concat([trip_equity_geog, df[['dpurp', 'Mode', 'equity_geog', 'trexpfac', 'Trip Type']]])


## regional mode share

In [5]:
df1 = trip_county.groupby(['Trip Type','Mode'],as_index=False)['trexpfac'].sum()
df2 = trip_county.groupby(['Trip Type'],as_index=False)['trexpfac'].sum()
df2.rename(columns={'trexpfac':'total_trips'}, inplace=True)

df = df1.merge(df2, on='Trip Type')
df['mode_share'] = df['trexpfac'] / df['total_trips']

df.pivot_table(columns='Mode', index='Trip Type', values='mode_share')

Mode,Bike,Drove Alone,Shared Ride,Transit,Walk
Trip Type,,,,,
All Trips,1.8%,40.0%,32.9%,4.8%,20.6%
Non-Work,1.8%,36.4%,35.8%,4.7%,21.3%
Work,1.8%,63.1%,14.3%,5.1%,15.7%


## trip mode share by home location

In [6]:
def calc_mode_share(df_trip, geog):
    
    # num trips by mode
    df1 = df_trip.groupby([geog, 'Trip Type', 'Mode'],as_index=False)['trexpfac'].sum()
    # num trips
    df2 = df_trip.groupby([geog, 'Trip Type'],as_index=False)['trexpfac'].sum()
    df2.rename(columns={'trexpfac':'total_trips'}, inplace=True)

    df = df1.merge(df2, on=[geog, 'Trip Type'])
    # mode share
    df['mode_share'] = df['trexpfac'] / df['total_trips']

    # mode share by trip type
    df_all = df.loc[df['Trip Type']=="All Trips"].pivot_table(columns='Mode', index=geog, values='mode_share')
    df_work = df.loc[df['Trip Type']=="Work"].pivot_table(columns='Mode', index=geog, values='mode_share')
    df_non_work = df.loc[df['Trip Type']=="Non-Work"].pivot_table(columns='Mode', index=geog, values='mode_share')
    # num trips by trip type
    trip_all = df.loc[df['Trip Type']=="All Trips"].pivot_table(columns='Mode', index=geog, values='trexpfac')
    trip_work = df.loc[df['Trip Type']=="Work"].pivot_table(columns='Mode', index=geog, values='trexpfac')
    trip_non_work = df.loc[df['Trip Type']=="Non-Work"].pivot_table(columns='Mode', index=geog, values='trexpfac')

    return df_all, df_work, df_non_work, trip_all, trip_work, trip_non_work


df_all, df_work, df_non_work, trip_all, trip_work, trip_non_work = calc_mode_share(trip_county,'hh_county')

In [7]:
df_all

Mode,Bike,Drove Alone,Shared Ride,Transit,Walk
hh_county,,,,,
King,2.0%,37.1%,30.4%,6.8%,23.8%
Kitsap,1.4%,44.7%,33.5%,2.9%,17.5%
Pierce,1.7%,42.3%,36.2%,1.8%,18.0%
Snohomish,1.5%,43.9%,35.6%,3.1%,15.8%


In [8]:
trip_all.style.format("{:0,.0f}")

Mode,Bike,Drove Alone,Shared Ride,Transit,Walk
hh_county,,,,,
King,"226,718","4,254,576","3,489,882","774,951","2,726,678"
Kitsap,"18,304","578,198","432,413","37,194","226,276"
Pierce,"75,728","1,857,434","1,591,777","80,341","788,932"
Snohomish,"69,180","1,963,312","1,591,188","137,202","706,901"


In [9]:
df_non_work

Mode,Bike,Drove Alone,Shared Ride,Transit,Walk
hh_county,,,,,
King,2.0%,33.6%,33.2%,6.8%,24.4%
Kitsap,1.4%,41.5%,36.3%,2.2%,18.6%
Pierce,1.7%,38.5%,39.2%,1.7%,18.8%
Snohomish,1.6%,39.9%,38.6%,3.0%,16.9%


In [10]:
trip_non_work.style.format("{:0,.0f}")

Mode,Bike,Drove Alone,Shared Ride,Transit,Walk
hh_county,,,,,
King,"194,986","3,300,471","3,268,583","672,105","2,397,862"
Kitsap,"16,105","467,593","408,647","25,199","209,137"
Pierce,"66,779","1,485,003","1,509,012","66,283","725,524"
Snohomish,"60,611","1,545,695","1,497,795","116,080","655,633"


In [11]:
df_work

Mode,Bike,Drove Alone,Shared Ride,Transit,Walk
hh_county,,,,,
King,1.9%,58.2%,13.5%,6.3%,20.1%
Kitsap,1.3%,66.7%,14.3%,7.2%,10.3%
Pierce,1.7%,68.8%,15.3%,2.6%,11.7%
Snohomish,1.4%,70.5%,15.8%,3.6%,8.7%


In [12]:
trip_work.style.format("{:0,.0f}")

Mode,Bike,Drove Alone,Shared Ride,Transit,Walk
hh_county,,,,,
King,"31,732","954,105","221,299","102,846","328,816"
Kitsap,"2,199","110,605","23,766","11,995","17,139"
Pierce,"8,949","372,431","82,765","14,058","63,408"
Snohomish,"8,569","417,617","93,393","21,122","51,268"


In [13]:
df_all, df_work, df_non_work, trip_all, trip_work, trip_non_work  = calc_mode_share(trip_rgc, 'hh_rgc')

df_all

Mode,Bike,Drove Alone,Shared Ride,Transit,Walk
hh_rgc,,,,,
All RGCs,3.2%,18.8%,17.1%,8.1%,52.8%
Auburn,2.6%,28.0%,26.8%,5.7%,37.0%
Bellevue,3.1%,10.0%,13.7%,9.2%,64.0%
Bothell Canyon Park,1.8%,41.9%,35.6%,2.5%,18.2%
Bremerton,2.9%,19.7%,16.8%,4.4%,56.3%
Burien,2.1%,32.8%,25.5%,5.6%,34.1%
Everett,3.0%,18.5%,17.6%,4.1%,56.8%
Federal Way,2.4%,26.8%,28.2%,7.5%,35.2%
Greater Downtown Kirkland,2.3%,39.3%,25.9%,9.0%,23.6%


In [14]:
trip_all.style.format("{:0,.0f}")

Mode,Bike,Drove Alone,Shared Ride,Transit,Walk
hh_rgc,,,,,
All RGCs,"114,081","673,515","614,160","289,508","1,893,247"
Auburn,"1,215","13,079","12,496","2,654","17,264"
Bellevue,"9,185","29,478","40,279","27,076","188,571"
Bothell Canyon Park,824,"19,724","16,743","1,167","8,562"
Bremerton,"1,230","8,338","7,101","1,854","23,826"
Burien,"1,578","24,622","19,123","4,196","25,600"
Everett,"5,347","33,432","31,838","7,338","102,500"
Federal Way,"1,416","15,767","16,583","4,404","20,701"
Greater Downtown Kirkland,"1,057","18,447","12,148","4,208","11,082"


In [15]:
df_non_work

Mode,Bike,Drove Alone,Shared Ride,Transit,Walk
hh_rgc,,,,,
All RGCs,3.2%,16.6%,18.6%,8.3%,53.3%
Auburn,2.4%,24.7%,28.5%,5.2%,39.1%
Bellevue,3.2%,8.7%,15.1%,9.9%,63.2%
Bothell Canyon Park,1.7%,38.1%,38.3%,2.5%,19.4%
Bremerton,2.8%,17.1%,17.9%,3.6%,58.6%
Burien,2.0%,28.0%,26.8%,5.8%,37.3%
Everett,2.8%,16.0%,18.9%,4.2%,58.1%
Federal Way,2.3%,23.2%,29.9%,7.4%,37.1%
Greater Downtown Kirkland,2.1%,35.4%,28.0%,9.1%,25.3%


In [16]:
trip_non_work.style.format("{:0,.0f}")

Mode,Bike,Drove Alone,Shared Ride,Transit,Walk
hh_rgc,,,,,
All RGCs,"97,342","502,506","565,220","253,253","1,616,371"
Auburn,988,"10,027","11,572","2,104","15,859"
Bellevue,"7,833","21,436","37,247","24,404","156,281"
Bothell Canyon Park,709,"15,701","15,806","1,031","8,003"
Bremerton,"1,020","6,285","6,557","1,333","21,537"
Burien,"1,337","18,348","17,569","3,803","24,451"
Everett,"4,350","24,754","29,334","6,514","90,124"
Federal Way,"1,197","11,935","15,401","3,820","19,076"
Greater Downtown Kirkland,842,"14,170","11,216","3,637","10,126"


In [17]:
df_work

Mode,Bike,Drove Alone,Shared Ride,Transit,Walk
hh_rgc,,,,,
All RGCs,3.0%,31.1%,8.9%,6.6%,50.4%
Auburn,3.7%,49.6%,15.0%,8.9%,22.8%
Bellevue,2.9%,17.0%,6.4%,5.6%,68.1%
Bothell Canyon Park,2.0%,69.7%,16.2%,2.4%,9.7%
Bremerton,3.7%,36.5%,9.7%,9.3%,40.8%
Burien,2.5%,65.3%,16.2%,4.1%,12.0%
Everett,3.9%,34.2%,9.9%,3.2%,48.8%
Federal Way,2.9%,51.5%,15.9%,7.8%,21.8%
Greater Downtown Kirkland,3.1%,61.5%,13.4%,8.2%,13.8%


In [18]:
trip_work.style.format("{:0,.0f}")

Mode,Bike,Drove Alone,Shared Ride,Transit,Walk
hh_rgc,,,,,
All RGCs,"16,739","171,009","48,940","36,255","276,876"
Auburn,227,"3,052",924,550,"1,405"
Bellevue,"1,352","8,042","3,032","2,672","32,290"
Bothell Canyon Park,115,"4,023",937,136,559
Bremerton,210,"2,053",544,521,"2,289"
Burien,241,"6,274","1,554",393,"1,149"
Everett,997,"8,678","2,504",824,"12,376"
Federal Way,219,"3,832","1,182",584,"1,625"
Greater Downtown Kirkland,215,"4,277",932,571,956


In [19]:
df_all, df_work, df_non_work, trip_all, trip_work, trip_non_work  = calc_mode_share(trip_rg, 'hh_rg_proposed')

df_all

Mode,Bike,Drove Alone,Shared Ride,Transit,Walk
hh_rg_proposed,,,,,
Cities and Towns,1.2%,46.2%,38.3%,1.7%,12.5%
Core Cities,1.8%,41.7%,34.8%,4.2%,17.6%
High Capacity Transit Communities,1.5%,44.6%,36.6%,3.8%,13.4%
Metropolitan Cities,2.5%,30.2%,24.4%,8.0%,34.9%
Rural Areas,0.9%,52.3%,39.7%,0.9%,6.2%
Urban Unincorporated Areas,1.2%,44.4%,43.8%,1.2%,9.4%


In [20]:
trip_all.style.format("{:0,.0f}")

Mode,Bike,Drove Alone,Shared Ride,Transit,Walk
hh_rg_proposed,,,,,
Cities and Towns,"20,847","782,264","648,452","28,597","211,337"
Core Cities,"90,704","2,152,502","1,797,986","216,887","907,706"
High Capacity Transit Communities,"70,859","2,139,739","1,754,543","184,495","642,717"
Metropolitan Cities,"179,424","2,150,067","1,737,184","571,949","2,482,882"
Rural Areas,"18,331","1,061,575","804,949","17,902","126,787"
Urban Unincorporated Areas,"9,766","367,397","362,166","9,858","77,358"


In [21]:
df_non_work

Mode,Bike,Drove Alone,Shared Ride,Transit,Walk
hh_rg_proposed,,,,,
Cities and Towns,1.3%,41.9%,41.6%,1.6%,13.5%
Core Cities,1.8%,37.6%,37.7%,4.2%,18.8%
High Capacity Transit Communities,1.5%,40.6%,39.7%,3.7%,14.5%
Metropolitan Cities,2.5%,27.2%,26.6%,8.2%,35.5%
Rural Areas,1.0%,48.6%,43.2%,0.7%,6.5%
Urban Unincorporated Areas,1.2%,40.8%,47.0%,1.1%,9.9%


In [22]:
trip_non_work.style.format("{:0,.0f}")

Mode,Bike,Drove Alone,Shared Ride,Transit,Walk
hh_rg_proposed,,,,,
Cities and Towns,"19,074","616,252","611,739","23,455","198,788"
Core Cities,"78,435","1,682,032","1,690,231","186,454","841,657"
High Capacity Transit Communities,"62,616","1,689,052","1,652,838","152,976","601,979"
Metropolitan Cities,"152,262","1,649,791","1,617,444","495,966","2,157,443"
Rural Areas,"17,070","859,872","764,446","12,952","115,009"
Urban Unincorporated Areas,"9,025","301,784","347,359","7,864","73,280"


In [23]:
df_work

Mode,Bike,Drove Alone,Shared Ride,Transit,Walk
hh_rg_proposed,,,,,
Cities and Towns,0.8%,74.7%,16.5%,2.3%,5.6%
Core Cities,1.8%,68.5%,15.7%,4.4%,9.6%
High Capacity Transit Communities,1.3%,71.2%,16.1%,5.0%,6.4%
Metropolitan Cities,2.6%,47.7%,11.4%,7.2%,31.0%
Rural Areas,0.5%,77.5%,15.6%,1.9%,4.5%
Urban Unincorporated Areas,0.8%,75.2%,17.0%,2.3%,4.7%


In [24]:
trip_work.style.format("{:0,.0f}")

Mode,Bike,Drove Alone,Shared Ride,Transit,Walk
hh_rg_proposed,,,,,
Cities and Towns,"1,773","166,012","36,713","5,142","12,549"
Core Cities,"12,269","470,470","107,755","30,433","66,049"
High Capacity Transit Communities,"8,243","450,687","101,705","31,519","40,738"
Metropolitan Cities,"27,162","500,276","119,740","75,983","325,439"
Rural Areas,"1,261","201,703","40,503","4,950","11,778"
Urban Unincorporated Areas,741,"65,613","14,807","1,994","4,078"


In [25]:
equity_geogs = util.summary_config['hh_equity_geogs']

trip_equity_geog = pd.DataFrame()
df_all = pd.DataFrame()
df_work = pd.DataFrame()
df_non_work = pd.DataFrame()
trip_work = pd.DataFrame()
trip_non_work = pd.DataFrame()

for geog in equity_geogs:
    df = prep_trip_df(pd.read_csv(util.output_path/ ('agg/dash/mode_share_'+geog+'.csv')))
    df['equity_geog'] = df[geog].map({
                                0: 'Below Regional Average', 
                                1: 'Above Regional Average', 
                                2: 'Higher Share of Equity Population'})
    df['EFA'] = geog
    # trip_equity_geog = pd.concat([trip_equity_geog, df[['equity_geog_type', 'dpurp', 'Mode', 'equity_geog','trexpfac', 'Trip Type']]])

    _df_all, _df_work, _df_non_work, _trip_all, _trip_work, _trip_non_work = calc_mode_share(df, 'equity_geog')

    _df_all['EFA'] = geog
    _df_work['EFA'] = geog
    _df_non_work['EFA'] = geog
    _trip_all['EFA'] = geog
    _trip_work['EFA'] = geog
    _trip_non_work['EFA'] = geog


    df_all = pd.concat([df_all, _df_all])
    df_work = pd.concat([df_work, _df_work])
    df_non_work = pd.concat([df_non_work, _df_non_work])
    trip_all = pd.concat([trip_all, _trip_all])
    trip_work = pd.concat([trip_work, _trip_work])
    trip_non_work = pd.concat([trip_non_work, _trip_non_work])

col_dict = {'hh_efa_dis': 'Disability',
                                'hh_efa_pov200': 'Income',
                                'hh_efa_poc': 'People of Color',
                                'hh_efa_lep': 'LEP',
                                'hh_efa_older': 'Older Adult',
                                'hh_efa_youth': 'Youth'}
col_list = ['EFA','Drove Alone', 'Shared Ride', 'Transit','Walk', 'Bike']

df_all['EFA'] = df_all['EFA'].map(col_dict)
df_all = df_all[col_list]
df_work['EFA'] = df_work['EFA'].map(col_dict)
df_work = df_work[col_list]
df_non_work['EFA'] = df_non_work['EFA'].map(col_dict)
df_non_work = df_non_work[col_list]
trip_all['EFA'] = trip_all['EFA'].map(col_dict)
trip_all = trip_all[col_list]
trip_work['EFA'] = trip_work['EFA'].map(col_dict)
trip_work = trip_work[col_list]
trip_non_work['EFA'] = trip_non_work['EFA'].map(col_dict)
trip_non_work = trip_non_work[col_list]

In [26]:
df_all

Mode,EFA,Drove Alone,Shared Ride,Transit,Walk,Bike
equity_geog,,,,,,
Above Regional Average,Disability,40.9%,33.4%,4.6%,19.3%,1.7%
Below Regional Average,Disability,41.0%,33.8%,5.0%,18.5%,1.7%
Higher Share of Equity Population,Disability,35.6%,29.2%,4.3%,28.8%,2.1%
Above Regional Average,Older Adult,42.4%,33.7%,4.0%,18.2%,1.6%
Below Regional Average,Older Adult,38.3%,32.4%,5.1%,22.3%,1.9%
Higher Share of Equity Population,Older Adult,41.5%,32.8%,4.9%,19.2%,1.7%
Above Regional Average,LEP,38.8%,32.8%,4.9%,21.6%,1.9%
Below Regional Average,LEP,40.9%,32.7%,4.4%,20.3%,1.7%
Higher Share of Equity Population,LEP,38.7%,33.4%,5.9%,20.1%,1.9%


In [27]:
df_non_work

Mode,EFA,Drove Alone,Shared Ride,Transit,Walk,Bike
equity_geog,,,,,,
Above Regional Average,Disability,37.3%,36.3%,4.5%,20.2%,1.7%
Below Regional Average,Disability,37.3%,36.9%,5.0%,19.1%,1.8%
Higher Share of Equity Population,Disability,32.3%,31.7%,4.2%,29.7%,2.1%
Above Regional Average,Older Adult,38.7%,36.8%,4.0%,18.9%,1.6%
Below Regional Average,Older Adult,34.6%,35.2%,5.1%,23.1%,2.0%
Higher Share of Equity Population,Older Adult,38.0%,35.5%,4.7%,20.1%,1.7%
Above Regional Average,LEP,35.2%,35.8%,4.9%,22.2%,1.9%
Below Regional Average,LEP,37.3%,35.6%,4.2%,21.1%,1.8%
Higher Share of Equity Population,LEP,34.9%,36.3%,6.0%,21.0%,1.9%


In [28]:
df_work

Mode,EFA,Drove Alone,Shared Ride,Transit,Walk,Bike
equity_geog,,,,,,
Above Regional Average,Disability,64.5%,14.7%,5.1%,13.9%,1.7%
Below Regional Average,Disability,64.3%,14.5%,5.3%,14.2%,1.6%
Higher Share of Equity Population,Disability,57.1%,13.3%,4.5%,23.0%,2.2%
Above Regional Average,Older Adult,65.8%,14.4%,4.3%,14.0%,1.5%
Below Regional Average,Older Adult,61.3%,14.2%,5.3%,17.2%,1.9%
Higher Share of Equity Population,Older Adult,64.2%,14.5%,6.4%,13.3%,1.6%
Above Regional Average,LEP,61.4%,14.2%,4.8%,17.8%,1.7%
Below Regional Average,LEP,63.8%,14.2%,5.2%,15.2%,1.7%
Higher Share of Equity Population,LEP,63.3%,15.1%,5.3%,14.3%,2.0%


## Mode Share by Trip Destination

In [29]:
tour_rgc_dest = pd.read_csv(util.output_path / 'agg/dash/tour_rgc_dest.csv')
trip_rgc_dest = pd.read_csv(util.output_path / 'agg/dash/trip_rgc_dest.csv')

# order RGCs

# all individual centers
l = tour_rgc_dest['tour_d_rgc'].sort_values().unique().tolist()
l.remove('Not in RGC')
l.remove(np.nan)

# aggregated centers
geog_order = ["Region","In RGC","Not in RGC"]

# combine
geog_order.extend(l)

In [30]:
def get_rgc_location(df, dest_col):
    # regional data
    df_region = df.copy()
    df_region[dest_col] = "Region"

    # aggregated centers
    df_isrgc = df.copy()
    df_isrgc.loc[df_isrgc[dest_col]!="Not in RGC", dest_col] = "In RGC"


    df_rgc = pd.concat([
        df_region,
        df_isrgc,
    # 2025 notes:
    # 7 records in the data are missing both mode and and RGC values
    # remove these records
        df.loc[~df[dest_col].isna()]
        ])

    df_rgc[dest_col] = pd.Categorical(df_rgc[dest_col], ordered=True,
                    categories=geog_order)
    
    return df_rgc

df_tour = get_rgc_location(tour_rgc_dest, 'tour_d_rgc')
df_trip = get_rgc_location(trip_rgc_dest, 'trip_d_rgc')




In [31]:
def calc_mode_share_by_rgc(df_trip_geog, geog_col, mode_col, n_trip_col):

    df = df_trip_geog.pivot_table(columns=mode_col, index=geog_col, values=n_trip_col, aggfunc='sum', observed=True)

    df = df.apply(lambda x: x/ df_trip_geog.groupby(geog_col, observed=True)[n_trip_col].sum())

    return df

\[TRIP\] Mode Share for All Purposes by Regional Center (Destination Location)

In [32]:
calc_mode_share_by_rgc(df_trip, 'trip_d_rgc', 'mode', 'trexpfac')

mode,Bike,HOV2,HOV3+,SOV,School Bus,TNC,Transit,Walk
trip_d_rgc,,,,,,,,
Region,1.8%,19.8%,12.5%,39.2%,1.4%,0.6%,4.7%,20.2%
In RGC,2.3%,13.6%,8.2%,27.7%,0.6%,1.4%,8.3%,37.9%
Not in RGC,1.6%,21.9%,13.9%,43.2%,1.6%,0.3%,3.4%,14.0%
Auburn,1.4%,20.5%,11.8%,40.4%,0.7%,0.3%,3.4%,21.5%
Bellevue,2.4%,9.9%,7.0%,18.3%,0.6%,0.8%,9.0%,52.0%
Bothell Canyon Park,1.4%,22.4%,13.4%,46.5%,1.1%,0.4%,1.3%,13.5%
Bremerton,2.2%,14.5%,8.1%,34.7%,0.4%,0.4%,4.0%,35.7%
Burien,1.4%,19.5%,11.2%,37.3%,1.0%,0.3%,3.6%,25.6%
Everett,2.1%,14.9%,9.1%,29.5%,0.7%,0.5%,3.8%,39.4%


\[TRIP\] Mode Share to Work by Regional Center (Destination Location)

In [33]:
df_trip_work = df_trip[df_trip['dpurp'] == 'Work'].copy()
calc_mode_share_by_rgc(df_trip_work, 'trip_d_rgc', 'mode', 'trexpfac')

mode,Bike,HOV2,HOV3+,SOV,School Bus,TNC,Transit,Walk
trip_d_rgc,,,,,,,,
Region,1.7%,8.8%,5.4%,62.4%,0.0%,1.2%,5.0%,15.5%
In RGC,2.0%,7.3%,4.5%,46.0%,0.0%,2.1%,9.0%,29.2%
Not in RGC,1.6%,9.8%,5.9%,73.5%,0.0%,0.5%,2.4%,6.3%
Auburn,1.4%,8.5%,5.7%,66.9%,NaN,0.3%,2.1%,14.9%
Bellevue,2.1%,6.3%,4.1%,37.4%,0.0%,1.0%,7.2%,41.9%
Bothell Canyon Park,1.5%,10.0%,6.3%,74.9%,NaN,0.4%,0.6%,6.2%
Bremerton,2.0%,8.9%,5.3%,62.4%,NaN,0.3%,5.0%,16.1%
Burien,1.5%,9.2%,5.1%,67.5%,NaN,0.5%,2.2%,14.0%
Everett,2.4%,8.0%,5.0%,53.7%,0.0%,0.5%,1.6%,28.7%


\[TRIP\] Mode Share for Non-work Purposes by Regional Center (Destination Location)

In [34]:
df_trip_non_work = df_trip[df_trip['dpurp'] != 'Work'].copy()
calc_mode_share_by_rgc(df_trip_non_work, 'trip_d_rgc', 'mode', 'trexpfac')

mode,Bike,HOV2,HOV3+,SOV,School Bus,TNC,Transit,Walk
trip_d_rgc,,,,,,,,
Region,1.8%,21.5%,13.6%,35.6%,1.6%,0.5%,4.6%,20.9%
In RGC,2.4%,15.3%,9.1%,22.8%,0.8%,1.2%,8.2%,40.2%
Not in RGC,1.6%,23.3%,14.9%,39.6%,1.8%,0.3%,3.5%,15.0%
Auburn,1.4%,22.1%,12.6%,36.8%,0.8%,0.3%,3.6%,22.4%
Bellevue,2.5%,11.0%,7.9%,12.6%,0.8%,0.7%,9.5%,55.0%
Bothell Canyon Park,1.4%,25.7%,15.3%,38.9%,1.4%,0.3%,1.5%,15.4%
Bremerton,2.2%,16.6%,9.2%,24.3%,0.5%,0.5%,3.6%,43.1%
Burien,1.4%,20.4%,11.7%,34.7%,1.0%,0.3%,3.8%,26.6%
Everett,2.1%,16.5%,10.0%,23.9%,0.9%,0.5%,4.3%,41.9%


## Mode Share by Tour Destination

\[TOUR\] Mode Share for All Purposes by Regional Center (Destination Location)

In [35]:
calc_mode_share_by_rgc(df_tour, 'tour_d_rgc', 'tmodetp', 'toexpfac')

tmodetp,Bike,HOV2,HOV3+,Park,SOV,School Bus,TNC,Transit,Walk
tour_d_rgc,,,,,,,,,
Region,1.6%,20.2%,16.6%,0.2%,33.5%,2.2%,0.8%,6.7%,18.2%
In RGC,1.9%,14.6%,11.4%,0.6%,26.8%,0.8%,1.7%,13.1%,29.2%
Not in RGC,1.5%,22.5%,18.8%,0.1%,36.3%,2.8%,0.4%,4.0%,13.6%
Auburn,0.8%,22.5%,17.3%,0.1%,36.7%,0.4%,0.3%,4.0%,17.9%
Bellevue,1.8%,10.9%,9.2%,0.6%,21.2%,0.6%,0.9%,13.9%,40.9%
Bothell Canyon Park,1.2%,22.8%,19.8%,0.0%,39.7%,2.1%,0.5%,1.9%,12.0%
Bremerton,1.7%,18.7%,13.3%,0.4%,36.2%,0.4%,0.6%,4.0%,24.8%
Burien,0.9%,21.0%,16.0%,0.1%,31.7%,1.5%,0.3%,4.7%,23.7%
Everett,1.8%,17.3%,13.7%,0.1%,27.8%,0.9%,0.6%,5.9%,32.0%


\[TOUR\] Mode Share to Work by Regional Center (Destination Location)

In [36]:
df_tour_work = df_tour[df_tour['pdpurp'] == 'Work'].copy()
calc_mode_share_by_rgc(df_tour_work, 'tour_d_rgc', 'tmodetp', 'toexpfac')

tmodetp,Bike,HOV2,HOV3+,Park,SOV,TNC,Transit,Walk
tour_d_rgc,,,,,,,,
Region,1.7%,13.5%,10.7%,0.8%,54.9%,2.0%,6.3%,10.0%
In RGC,2.0%,10.6%,8.6%,1.5%,41.5%,3.6%,11.6%,20.6%
Not in RGC,1.5%,15.4%,12.2%,0.4%,64.2%,0.9%,2.6%,2.8%
Auburn,1.4%,14.7%,12.5%,0.2%,58.0%,0.6%,3.1%,9.5%
Bellevue,2.1%,9.2%,7.9%,1.3%,35.8%,1.6%,9.4%,32.6%
Bothell Canyon Park,1.5%,15.6%,13.1%,0.1%,65.4%,0.8%,0.9%,2.6%
Bremerton,1.7%,14.4%,11.0%,0.9%,56.2%,0.6%,5.8%,9.3%
Burien,1.4%,15.2%,10.9%,0.4%,59.9%,0.9%,2.5%,8.7%
Everett,2.6%,13.2%,10.8%,0.2%,48.0%,1.0%,2.0%,22.3%


\[TOUR\] Mode Share for Non-work Purposes by Regional Center (Destination Location)

In [37]:
df_tour_non_work = df_tour[df_tour['pdpurp'] != 'Work'].copy()
calc_mode_share_by_rgc(df_tour_non_work, 'tour_d_rgc', 'tmodetp', 'toexpfac')

tmodetp,Bike,HOV2,HOV3+,SOV,School Bus,TNC,Transit,Walk
tour_d_rgc,,,,,,,,
Region,1.6%,22.8%,18.9%,25.1%,3.0%,0.3%,6.9%,21.4%
In RGC,1.8%,17.1%,13.1%,17.8%,1.3%,0.5%,14.0%,34.5%
Not in RGC,1.5%,24.7%,20.9%,27.6%,3.6%,0.3%,4.5%,16.9%
Auburn,0.6%,24.8%,18.7%,30.4%,0.6%,0.2%,4.3%,20.4%
Bellevue,1.5%,12.3%,10.2%,9.7%,1.1%,0.3%,17.4%,47.5%
Bothell Canyon Park,1.0%,27.7%,24.3%,22.3%,3.5%,0.3%,2.5%,18.3%
Bremerton,1.7%,21.9%,14.9%,21.3%,0.6%,0.5%,2.6%,36.4%
Burien,0.9%,22.0%,16.9%,26.6%,1.8%,0.2%,5.1%,26.5%
Everett,1.4%,19.1%,15.1%,18.4%,1.3%,0.4%,7.8%,36.6%
